# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import sys
import joblib 
import numpy as np
import pandas as pd
import nltk

nltk.download(['punkt', 'wordnet', 'stopwords'])
  
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

#!pip install ipynb
#from ipynb.fs.full.workspace_utils import active_session

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prasa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prasa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prasa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data from database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Messages', con = engine)
X = df['message']
Y = df[df.columns[5:]]

In [5]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [6]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [7]:
for c in Y.columns:
    if len(Y[c].unique()) > 2:
        print(c)

In [8]:
Y.head()

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [9]:
'''
This function accepts text message as input.
Then normalize the text, remove punctuation, tokenize, remove stop words, reduce words to their root form
'''

def tokenize(text):
     
    #Normalize text & remove punctuation
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    #Tokenize text, remove stop words
    tokens = word_tokenize(text)
    stop_words = stopwords.words("english")
    words = [w for w in tokens if w not in stop_words]
        
   
    # Reduce words to their root form
    lemmatizer = WordNetLemmatizer()
    lemmed = [lemmatizer.lemmatize(w) for w in words]
    
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier((RandomForestClassifier())))
        ])


In [14]:
sorted(pipeline.get_params().keys())

['clf',
 'clf__estimator',
 'clf__estimator__bootstrap',
 'clf__estimator__ccp_alpha',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__max_samples',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'clf__estimator__oob_score',
 'clf__estimator__random_state',
 'clf__estimator__verbose',
 'clf__estimator__warm_start',
 'clf__n_jobs',
 'memory',
 'steps',
 'tfidf',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'vect',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
# Split Train & Test data
X_train, X_test, y_train, y_test = train_test_split(X,Y)

# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001C4C086BAF0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
y_pred = pipeline.predict(X_test)

In [57]:
y_pred.shape,y_test.shape,len(list(Y.columns))

((6529, 35), (6529, 35), 35)

In [17]:
print(classification_report(y_test, y_pred, target_names=Y.columns))

                        precision    recall  f1-score   support

               request       0.82      0.48      0.61      1124
                 offer       0.00      0.00      0.00        38
           aid_related       0.75      0.69      0.72      2716
          medical_help       0.58      0.08      0.14       490
      medical_products       0.75      0.11      0.19       327
     search_and_rescue       0.73      0.04      0.08       183
              security       0.00      0.00      0.00       116
              military       0.71      0.05      0.09       219
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.43      0.57       434
                  food       0.83      0.61      0.70       744
               shelter       0.78      0.33      0.46       559
              clothing       0.88      0.13      0.23       108
                 money       0.62      0.03      0.06       154
        missing_people       1.00      

C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_cla

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
parameters = {
    'clf__estimator__criterion': ['entropy'],
    'clf__estimator__max_depth': [2, 5],
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [19]:
cv

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001C4C086BAF0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__criterion': ['entropy'],
                         'clf__estimator__max_depth': [2, 5]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001C4C086BAF0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__criterion': ['entropy'],
                         'clf__estimator__max_depth': [2, 5]})

In [21]:
y_pred = cv.predict(X_test)

In [22]:
print(classification_report(y_test, y_pred, target_names=Y.columns))

                        precision    recall  f1-score   support

               request       0.00      0.00      0.00      1472
                 offer       0.00      0.00      0.00        35
           aid_related       1.00      0.00      0.00      3583
          medical_help       0.00      0.00      0.00       675
      medical_products       0.00      0.00      0.00       416
     search_and_rescue       0.00      0.00      0.00       223
              security       0.00      0.00      0.00       162
              military       0.00      0.00      0.00       280
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       542
                  food       0.00      0.00      0.00       919
               shelter       0.00      0.00      0.00       775
              clothing       0.00      0.00      0.00       124
                 money       0.00      0.00      0.00       202
        missing_people       0.00      

C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [23]:
joblib.dump(cv.best_estimator_, 'random_forest_best.pkl')

['random_forest_best.pkl']

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:
pipeline = Pipeline([('vectorizer', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced')))])

In [25]:
%%time
parameters = {'vectorizer__ngram_range': [(1, 1), (1, 2),(2,2)],
               'tfidf__use_idf': (True, False)}
gs_clf = GridSearchCV(pipeline, param_grid=parameters)

Wall time: 0 ns


In [26]:
gs_clf = gs_clf.fit(X_train, y_train)

y_pred = gs_clf.predict(X_test)
#with active_session():
    


C:\Users\prasa\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 8 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
C:\Users\prasa\

In [27]:
print(classification_report(y_test, y_pred, target_names=Y.columns))

                        precision    recall  f1-score   support

               request       0.69      0.57      0.62      1472
                 offer       0.00      0.00      0.00        35
           aid_related       0.69      0.62      0.65      3583
          medical_help       0.47      0.25      0.32       675
      medical_products       0.49      0.30      0.37       416
     search_and_rescue       0.49      0.13      0.21       223
              security       0.14      0.03      0.05       162
              military       0.43      0.14      0.21       280
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.63      0.67       542
                  food       0.75      0.58      0.66       919
               shelter       0.63      0.42      0.51       775
              clothing       0.41      0.19      0.26       124
                 money       0.38      0.14      0.20       202
        missing_people       0.50      

C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [28]:
joblib.dump(gs_clf.best_estimator_, 'onevsrest_linear_best.pkl')

['onevsrest_linear_best.pkl']

## Testing Another Model

In [29]:
%%time

pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(BernoulliNB()))])


Wall time: 1 ms


In [30]:
parameters = {'vect__max_features': (None, 5000),
              'tfidf__use_idf': (True, False)}

In [31]:
gs_clf = GridSearchCV(pipeline, param_grid=parameters)

In [32]:
gs_clf = gs_clf.fit(X_train, y_train)

y_pred = gs_clf.predict(X_test)

In [33]:
print(classification_report(y_test, y_pred, target_names=Y.columns))

                        precision    recall  f1-score   support

               request       0.76      0.50      0.60      1472
                 offer       0.00      0.00      0.00        35
           aid_related       0.72      0.59      0.65      3583
          medical_help       0.42      0.12      0.18       675
      medical_products       0.38      0.08      0.13       416
     search_and_rescue       0.20      0.03      0.05       223
              security       0.00      0.00      0.00       162
              military       0.24      0.04      0.06       280
           child_alone       0.00      0.00      0.00         0
                 water       0.44      0.08      0.13       542
                  food       0.55      0.14      0.22       919
               shelter       0.50      0.12      0.20       775
              clothing       0.00      0.00      0.00       124
                 money       0.15      0.02      0.04       202
        missing_people       0.07      

C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prasa\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [34]:
joblib.dump(gs_clf.best_estimator_, 'bernoulli_best.pkl')

['bernoulli_best.pkl']

### 9. Export your model as a pickle file

In [35]:
joblib.dump(gs_clf.best_estimator_, 'onevsrest_linear_best.pkl')

['onevsrest_linear_best.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.